# RGBD camera simulation in Drake
This notebook includes the boilerplate code needed to run RGBD camera simulation in Drake. It assumes familiarity with `MultibodyPlant` and [constructing and running simulations with digram systems](https://nbviewer.jupyter.org/github/RobotLocomotion/drake/blob/master/tutorials/dynamical_systems.ipynb).

In [ ]:
import numpy as np
import os
from pydrake.systems.sensors import RgbdSensor, PixelType
from pydrake.multibody.plant import MultibodyPlant

from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.perception import DepthImageToPointCloud, BaseField
from pydrake.systems.framework import DiagramBuilder, AbstractValue
from pydrake.geometry.render import (CameraProperties, DepthCameraProperties,
                                     MakeRenderEngineVtk, RenderEngineVtkParams)
from pydrake.systems.analysis import Simulator
import meshcat
from pydrake.systems.meshcat_visualizer import (MeshcatVisualizer,
    MeshcatPointCloudVisualizer, AddTriad)


def RenderSystemWithGraphviz(system, output_file="system_view.gz"):
    """ Renders the Drake system (presumably a diagram,
    otherwise this graph will be fairly trivial) using
    graphviz to a specified file. """
    from graphviz import Source
    string = system.GetGraphvizString()
    src = Source(string)
    src.render(output_file, view=False)

In [ ]:
# First start meshcat for visualization - this only has to be run once.

# If you interrupt the kernel of this notebook, you'll need to run this cell again to 
# restart the meshcat server, and then refresh the visualization window. 

# This will open a mesh-cat server in the background, 
# click on the url to display visualization in a separate window. 
vis = meshcat.Visualizer()

In [ ]:
# Run this if you feel the visualizer is messed up and want to start over.
vis.delete()

In [ ]:
# Construct a drake simulation diagram builder.
builder = DiagramBuilder()

# Construct empty MultibodyPlant.
plant = MultibodyPlant(2e-3)
_, scene_graph = AddMultibodyPlantSceneGraph(builder, plant=plant)
parser = Parser(plant=plant, scene_graph=scene_graph)
plant.mutable_gravity_field().set_gravity_vector([0, 0, 0])

# Add iiwa to MultibodyPlant
robot_sdf_path = FindResourceOrThrow(
    "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
parser.AddModelFromFile(robot_sdf_path)
plant.WeldFrames(A=plant.world_frame(),
                 B=plant.GetFrameByName("iiwa_link_0"))

# Add a box to MultibodyPlant as the ground.
ground_sdf_path = os.path.join(
    os.getcwd(), "ground_box.sdf")
parser.AddModelFromFile(ground_sdf_path)

# Define pose of the box in world frame.
X_WB = RigidTransform.Identity()
X_WB.set_translation([0, -(2.5 - np.sqrt(2) - 0.1), -0.6])  # "ground"
plant.WeldFrames(A=plant.world_frame(),
                 B=plant.GetFrameByName("ground"),
                 X_AB=X_WB)

# Finalize MultibodyPlant.
plant.Finalize()

In [ ]:
# Camera instrinsics
width = 640
height = 480
fov_y = np.pi / 2
focal_y = height / 2 / np.tan(fov_y / 2)
focal_x = focal_y
center_x = width / 2 - 0.5
center_y = height / 2 - 0.5
intrinsic_matrix = np.array([
    [focal_x, 0, center_x],
    [0, focal_y, center_y],
    [0, 0, 1]])

default_renderer_name = "manip_station_renderer"

# RGB camera properties
color_properties = CameraProperties(
    width=width, height=height, fov_y=fov_y,
    renderer_name=default_renderer_name)

# Depth camera properties
depth_properties = DepthCameraProperties(
    width=width, height=height, fov_y=fov_y,
    renderer_name=default_renderer_name, z_near=0.1, z_far=5.5)

# Camera extrinsics
X_WC = RigidTransform()
X_WC.set_translation(np.array([1, 0, 0]))
X_WC.set_rotation(RollPitchYaw(0, -np.pi/2, 0).ToRotationMatrix())

# Construct camera
camera = RgbdSensor(parent_id=scene_graph.world_frame_id(), X_PB=X_WC,
                    color_properties=color_properties,
                    depth_properties=depth_properties,
                    show_window=False)

In [ ]:
# Add camera to diagram.
builder.AddSystem(camera)
builder.Connect(scene_graph.get_query_output_port(),
                camera.query_object_input_port())

scene_graph.AddRenderer(default_renderer_name,
                        MakeRenderEngineVtk(RenderEngineVtkParams()))

# Add [camera -> point cloud] system to diagram.
di2pc = builder.AddSystem(DepthImageToPointCloud(
            camera.depth_camera_info(), PixelType.kDepth16U, 1e-3,
            fields=BaseField.kXYZs | BaseField.kRGBs))
builder.Connect(
    camera.GetOutputPort("depth_image_16u"),
    di2pc.depth_image_input_port())
builder.Connect(
    camera.GetOutputPort("color_image"),
    di2pc.color_image_input_port())

# Add meshcat visualizer to diagram.
vis = builder.AddSystem(MeshcatVisualizer(
    scene_graph, zmq_url="tcp://127.0.0.1:6000"))

builder.Connect(scene_graph.GetOutputPort("lcm_visualization"),
                vis.get_input_port(0))
scene_pc_vis = builder.AddSystem(MeshcatPointCloudVisualizer(
    vis, name="scene_point_cloud"))
builder.Connect(di2pc.GetOutputPort("point_cloud"),
                scene_pc_vis.GetInputPort("point_cloud_P"))

# Complete diagram construction.
diagram = builder.Build()

# Render diagram structure in a PDF file (saved in the directory of this notebook).
RenderSystemWithGraphviz(diagram)

In [ ]:
# Add diagram to a simulator.
simulator = Simulator(diagram)

plant_context = diagram.GetMutableSubsystemContext(
    plant, simulator.get_mutable_context())

# set initial state of the robot
plant_context.FixInputPort(
    plant.GetInputPort("iiwa7_actuation").get_index(), np.zeros(7))

di2pc_context = diagram.GetSubsystemContext(
    di2pc, simulator.get_context())

di2pc_context.FixInputPort(
    di2pc.GetInputPort("camera_pose").get_index(),
    AbstractValue.Make(X_WC))

simulator.AdvanceTo(0.2)

In [ ]:
# Draw camera frame
name = "camera"
prefix = "cameras"
AddTriad(vis.vis, name="camera", prefix="cameras", length=0.15, radius=0.006)
vis.vis[prefix][name].set_transform(X_WC.matrix())

In [ ]:
# get point cloud from output port
pc = di2pc.GetOutputPort("point_cloud").Eval(di2pc_context)

color = pc.rgbs()
position = pc.xyzs()